In [ ]:
import cv2
import matplotlib.pyplot as plt
from utils.image_handler import preprocess_img
%matplotlib inline

img = cv2.cvtColor(cv2.imread('./test_images/stop_1405106680.avi_image9.png'), cv2.COLOR_BGR2RGB)
# img = cv2.cvtColor(cv2.imread('./test_images/stop.jpg'), cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (int(1280/2),int(960/2)))
plt.figure(figsize=(10, 10))
plt.imshow(img)
print(img.shape)

In [ ]:
import numpy as np
import cv2
# dog    = Box(70, 258, 209, 356, 'Dog', 0.79)
# person = Box(190, 98, 271, 379, 'Person', 0.81)
# horse  = Box(399, 128, 605, 352, 'Horse', 0.89)
# boxes = [dog, person, horse]
class Box():
    def __init__(self, x1, y1, x2, y2, label, score):
        self.x1 = int(x1)
        self.x2 = int(x2)
        self.y1 = int(y1)
        self.y2 = int(y2)
        self.cls = str(label)
        self.score = float(score)

        
def draw(img, boxes, color=[0, 255, 0], thickness=1):
    result = np.copy(img)
    
    for box in boxes:
        p1 = (box.x1, box.y1)
        p2 = (box.x2, box.y2)
        result = cv2.rectangle(result, p1, p2, color=(0, 255, 0), thickness=thickness)
        
    return result

In [ ]:
import tensorflow as tf
import keras.backend as K
from keras.models import load_model, Model
from keras.layers import Conv2D
from keras.regularizers import l2
from cfg import *
from model.yolov2 import YOLOv2, darknet19

K.clear_session()  # to avoid duplicating model
THRESHOLD = 0.7 
# N_CLASSES = 80
# ANCHORS   = np.array(((0.57273, 0.677385), (1.87446, 2.06253), 
#                       (3.33843, 5.47434), (7.88282, 3.52778), (9.77052, 9.16828)))

with tf.device('/cpu:0'):
    yolov2 = YOLOv2(feature_extractor=darknet19(input_size=(None, None, 3)), num_anchors=N_ANCHORS, num_classes=N_CLASSES)
    model = yolov2.model
    model.load_weights('../backup/yolov2-epoch9-loss3.57268298149.weights')
#     model.load_weights('../yolo-coco.h5')

# model.summary()


In [ ]:
# Preprocess input
input_img = img / 255. 
input_img = np.expand_dims(input_img, 0)

# Making prediction
netout = model.predict(input_img)

netshape = np.shape(netout)
netout = np.reshape(netout, [-1, netshape[1], netshape[2], 5, N_CLASSES+5])

GRID_H, GRID_W = netout.shape[1:3]
height, width ,c  = img.shape
print(np.shape(netout))

In [ ]:
image_shape = img.shape
ANCHORS = ANCHORS * np.array([GRID_W, GRID_H]) / 2.0
# Create GRID-cell map
cx = tf.cast((K.arange(0, stop=GRID_W)), dtype=tf.float32)
cx = K.tile(cx, [GRID_H])
cx = K.reshape(cx, [-1, GRID_H, GRID_W, 1])

cy = K.cast((K.arange(0, stop=GRID_H)), dtype=tf.float32)
cy = K.reshape(cy, [-1, 1])
cy = K.tile(cy, [1, GRID_W])  
cy = K.reshape(cy, [-1])    
cy = K.reshape(cy, [-1, GRID_H, GRID_W, 1])

c_xy = tf.stack([cx, cy], -1)
c_xy = K.cast(c_xy, K.dtype(netout))


anchors_tensor = tf.cast(K.reshape(K.variable(ANCHORS), [1, 1, 1, 5, 2]), tf.float32)
netout_size   = K.cast(K.reshape([GRID_W, GRID_H], [1, 1, 1, 1, 2]), K.dtype(netout))

box_xy          = K.sigmoid(netout[..., :2])
box_wh          = K.exp(netout[..., 2:4])
box_confidence  = K.sigmoid(netout[..., 4:5])
box_class_probs = K.softmax(netout[..., 5:])

# Adjust predictions to each spatial grid point and anchor size.
box_xy = (box_xy + c_xy) / netout_size
box_wh = box_wh * anchors_tensor / netout_size
box_mins  = box_xy - (box_wh / 2.)
box_maxes = box_xy + (box_wh / 2.)

# Y1, X1, Y2, X2
boxes   = K.concatenate([box_mins[..., 1:2], box_mins[..., 0:1],  box_maxes[..., 1:2], box_maxes[..., 0:1]])

box_scores       = box_confidence * box_class_probs
box_classes      = K.argmax(box_scores, -1)
box_class_scores = K.max(box_scores, -1)
prediction_mask  = (box_class_scores >= 0.1)

boxes   = tf.boolean_mask(boxes, prediction_mask)
scores  = tf.boolean_mask(box_class_scores, prediction_mask)
classes = tf.boolean_mask(box_classes, prediction_mask)

# Scale boxes back to original image shape.
height = image_shape[0]
width = image_shape[1]

image_dims = tf.cast(K.stack([height, width, height, width]), tf.float32)
image_dims = K.reshape(image_dims, [1, 4])
boxes = boxes * image_dims

nms_index = tf.image.non_max_suppression(boxes, scores, tf.Variable(10), iou_threshold=0.6)
boxes = K.gather(boxes, nms_index)
scores = K.gather(scores, nms_index)
classes = K.gather(classes, nms_index)


with tf.Session() as sess:   
    tf.global_variables_initializer().run()
#     obj_score = box_scores.eval()
#     print(obj_score[obj_score > 0.01])
    boxes_prediction = boxes.eval()
    scores_prediction = scores.eval()
    classes_prediction = classes.eval()

print(boxes_prediction, scores_prediction, classes_prediction)

In [ ]:
result = boxes_prediction

bboxes = []
for box, score, cls in zip(result, scores_prediction, classes_prediction):
    y1, x1, y2, x2 = box.astype(np.int)
    bboxes.append(Box(x1, y1, x2, y2, cls, score))

result = draw(img, bboxes)
plt.figure(figsize=(15, 15))

plt.imshow(result)
print(img.shape)